In [26]:
from __future__ import annotations
import math
import numpy as np

In [69]:
class Value:
    def __init__(self, data: float, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_children)
        self._backward = lambda: None

    def __repr__(self):
        return f'Value(data={self.data}, grad={self.grad})'

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data + out.grad
        out._backward = _backward
        return out

    def tanh(self):
        y = math.tanh(self.data)
        out = Value(y, (self,), 'tanh')

        def _backward():
            self.grad += (1 - y*y) + out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)); 'Only accepting int, float'
        out = Value(self.data ** other, (self, ), '**') 
        
        def _backward():
            self.grad += other * (self.data) ** (other - 1) * out.grad

        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def backward(self):
        visited = set()
        topo = []

        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)

        build(self)
        self.grad = 1.0
        for x in reversed(topo):
            x._backward()
                
    

In [54]:
a = Value(2)
c = Value(3)
b = Value(4)

d = a * b
f = d + c
f

Value(data=11, grad=0.0)

In [55]:
f.grad = 1

In [56]:
f._backward()
d

Value(data=8, grad=1.0)

In [57]:
d._backward()
d

Value(data=8, grad=1.0)

In [58]:
c._backward()

In [59]:
d._backward()

In [60]:
a._backward()
b._backward()

In [61]:
a,b,c,d,f # checking populations of grads

(Value(data=2, grad=8.0),
 Value(data=4, grad=4.0),
 Value(data=3, grad=1.0),
 Value(data=8, grad=1.0),
 Value(data=11, grad=1))

In [62]:
d._prev

{Value(data=2, grad=8.0), Value(data=4, grad=4.0)}

In [63]:
c = Value(3)
x = c ** 4 
x

Value(data=81, grad=0.0)

In [64]:
x.grad = 1
x._backward()
c._backward()
c

Value(data=3, grad=108.0)

In [65]:
x._prev

{Value(data=3, grad=108.0)}

In [66]:
a = Value(3)
b = Value(4)
c = Value(5)

d = a * c
e = d + 2

f = d + c
f

Value(data=20, grad=0.0)

In [67]:
f.backward()

In [70]:
# lets try a simple neuron

# inputs
x1 = Value(2)
x2 = Value(0)

# weights
w1 = Value(-3)
w2 = Value(1)

# biases
b2 = Value(6.8)

# wx+b
w1x1 = w1 * x1
w2x2 = w2 * x2
w1x1w2x2 = w1x1 + w2x2
n = w1x1w2x2 + b
o = n.tanh()

TypeError: must be real number, not Value

In [73]:
# df/ df = 1
# df / dd = d(d+c)/dd = 

__main__.Value